In [28]:
from pyspark.sql import SparkSession
jdbc_driver_path = "/util/jar/postgresql-42.5.1.jar"

In [31]:
spark = SparkSession.builder \
    .appName("Escrita no PostgreSQL") \
    .config("spark.jars", jdbc_driver_path) \
    .getOrCreate()

url = "jdbc:postgresql://db:5432/admin"
properties = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}
tabela = "PR_VL_MD_TO_CGDA"



In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, percentile_approx

spark = (SparkSession.builder
             .appName('lab') # Name the app
             .config("hive.metastore.uris", "thrift://metastore:9083") # Set external Hive Metastore
             .config("hive.metastore.schema.verification", "false") # Prevent some errors
             .config("spark.sql.repl.eagerEval.enabled", True)
             .config('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension') \
            .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.delta.catalog.DeltaCatalog') \
             .enableHiveSupport()
             .getOrCreate())

In [2]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|    teste|
+---------+



In [5]:
trusted_path_previsao_chegada = 's3a://trusted/previsao_chegada'

In [6]:
df_previsao_chegada = spark.read.format('delta').load(trusted_path_previsao_chegada)

In [7]:
df_previsao_chegada.show()

+-------------+---------+-----------------------+---------------+------------+----------+-------------------+-------------------+------------------+--------------+
|dat_ref_carga|id_parada|horario_previsto_cheada|PREFIXO_VEICULO|latitude_loc|longtitude|   LATITUDE_VEICULO|  LONGITUDE_VEICULO|      distancia_km|diff_in_minute|
+-------------+---------+-----------------------+---------------+------------+----------+-------------------+-------------------+------------------+--------------+
|        20:47|360004796|                  20:53|          31943|  -46.444853|-23.513681| -46.46042166666666| -23.51553333333333|1.7369629819857777|           6.0|
|        20:47|360004796|                  21:03|          31936|  -46.444853|-23.513681|-46.485756666666674|-23.515643333333333| 4.550762764692933|          16.0|
|        20:47|360004796|                  21:04|          31840|  -46.444853|-23.513681|-46.490275000000004|         -23.515145| 5.051940356784393|          17.0|
|        20:47|3

In [11]:
df_insight_velocidade = df_previsao_chegada.withColumn("velocidade_para_chegar", (col("distancia_km") * 60) / col("diff_in_minute"))

In [21]:
from pyspark.sql.functions import percentile_approx
df_insight_velocidade.filter("velocidade_para_chegar > 40").groupBy("id_parada").agg(percentile_approx("velocidade_para_chegar", 0.5).alias("media_velocidade"))

id_parada,media_velocidade
1010076,191.9065847677062
1410076,49.323195284355265
1506163,121.39152707090088
3514897,73.94280743889519
4203724,105.86262687488104
5614544,99.42775294780941
5614560,130.09091261999896
5614562,79.85953522343779
6009703,243.49736473422357
7014417,285.96742041965507


In [26]:
spark.sql("CREATE DATABASE IF NOT EXISTS SPTRANS")

""


In [27]:
df_insight_velocidade.write.mode("overwrite").saveAsTable("SPTRANS.PR_VL_MD_TO_CGDA")

In [32]:
df_insight_velocidade.write.jdbc(url=url, table=tabela, mode="overwrite", properties=properties)